In [1]:
import pandas as pd

books = pd.read_csv('books_with_categories.csv')


In [9]:
from transformers import pipeline
classifier = pipeline("text-classification",
                       model="j-hartmann/emotion-english-distilroberta-base",
                       top_k = None,
                       device = "mps"
                         )
classifier("i won !")


Device set to use mps


[[{'label': 'joy', 'score': 0.9457526206970215},
  {'label': 'surprise', 'score': 0.03851403668522835},
  {'label': 'anger', 'score': 0.00627565523609519},
  {'label': 'neutral', 'score': 0.004466700833290815},
  {'label': 'sadness', 'score': 0.0039158654399216175},
  {'label': 'fear', 'score': 0.0006540586473420262},
  {'label': 'disgust', 'score': 0.0004209853068459779}]]

In [11]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [10]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548413634300232},
  {'label': 'neutral', 'score': 0.1698521375656128},
  {'label': 'sadness', 'score': 0.11640870571136475},
  {'label': 'surprise', 'score': 0.020700665190815926},
  {'label': 'disgust', 'score': 0.01910073310136795},
  {'label': 'joy', 'score': 0.015161280520260334},
  {'label': 'anger', 'score': 0.0039351508021354675}]]

In [12]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296023964881897},
  {'label': 'neutral', 'score': 0.14038589596748352},
  {'label': 'fear', 'score': 0.06816217303276062},
  {'label': 'joy', 'score': 0.04794251546263695},
  {'label': 'anger', 'score': 0.009156357496976852},
  {'label': 'disgust', 'score': 0.002628477755934},
  {'label': 'sadness', 'score': 0.0021221647039055824}],
 [{'label': 'neutral', 'score': 0.4493715167045593},
  {'label': 'disgust', 'score': 0.27359068393707275},
  {'label': 'joy', 'score': 0.10908294469118118},
  {'label': 'sadness', 'score': 0.09362741559743881},
  {'label': 'anger', 'score': 0.040478210896253586},
  {'label': 'surprise', 'score': 0.026970192790031433},
  {'label': 'fear', 'score': 0.006879056338220835}],
 [{'label': 'neutral', 'score': 0.6462160348892212},
  {'label': 'sadness', 'score': 0.2427334487438202},
  {'label': 'disgust', 'score': 0.0434226393699646},
  {'label': 'surprise', 'score': 0.02830049954354763},
  {'label': 'joy', 'score': 0.014211451634

In [13]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)



In [14]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [15]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296023964881897},
 {'label': 'neutral', 'score': 0.14038589596748352},
 {'label': 'fear', 'score': 0.06816217303276062},
 {'label': 'joy', 'score': 0.04794251546263695},
 {'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.002628477755934},
 {'label': 'sadness', 'score': 0.0021221647039055824}]

In [17]:
sorted(predictions[0], key=lambda x: x['label'])

[{'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.002628477755934},
 {'label': 'fear', 'score': 0.06816217303276062},
 {'label': 'joy', 'score': 0.04794251546263695},
 {'label': 'neutral', 'score': 0.14038589596748352},
 {'label': 'sadness', 'score': 0.0021221647039055824},
 {'label': 'surprise', 'score': 0.7296023964881897}]

In [21]:

import numpy as np

# Define list of possible emotions we want to analyze
emotion_labels = ["joy", "sadness", "anger", "fear", "surprise", "disgust", "neutral"]

# Initialize empty list for ISBN numbers
isbn = []

# Create a dictionary with empty lists for each emotion to store scores
# Format: {"joy": [], "sadness": [], ...}
emotion_scores = {label : [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    """
    Calculate the maximum score for each emotion across all predictions
    
    Args:
        predictions: List of emotion predictions, where each prediction is a list of 
                    dictionaries containing 'label' and 'score' for each emotion
    
    Returns:
        Dictionary with maximum score for each emotion
    """
    # Initialize dictionary to store scores for each emotion
    per_emotion_scores = {label : [] for label in emotion_labels}
    
    # Iterate through each prediction set
    for prediction in predictions:
        # Sort predictions by emotion label to ensure consistent ordering
        sorted_predictions = sorted(prediction, key=lambda x: x['label'])
        
        # For each emotion, extract its score and append to corresponding list
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]['score'])
    
    # Calculate maximum score for each emotion across all predictions
    # Returns: {"joy": max_score, "sadness": max_score, ...}
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [23]:
#testing for first 10 books
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])


In [24]:
emotion_scores

{'joy': [0.06413353234529495,
  0.6126179099082947,
  0.06413353234529495,
  0.35148343443870544,
  0.08141229301691055,
  0.2322252243757248,
  0.5381841659545898,
  0.06413353234529495,
  0.3006708025932312,
  0.06413353234529495],
 'sadness': [0.27359068393707275,
  0.34828516840934753,
  0.1040065586566925,
  0.15072256326675415,
  0.1844949573278427,
  0.7271744608879089,
  0.15585489571094513,
  0.1040065586566925,
  0.27948132157325745,
  0.17792607843875885],
 'anger': [0.9281686544418335,
  0.9425276517868042,
  0.9723208546638489,
  0.36070677638053894,
  0.09504333138465881,
  0.051362719386816025,
  0.747428297996521,
  0.40449604392051697,
  0.9155239462852478,
  0.051362719386816025],
 'fear': [0.9327977299690247,
  0.7044217586517334,
  0.7672373056411743,
  0.2518809139728546,
  0.04056433215737343,
  0.04337581247091293,
  0.8725650906562805,
  0.04056433215737343,
  0.04056433215737343,
  0.04056433215737343],
 'surprise': [0.6462160348892212,
  0.8879395127296448,
  

In [27]:
from tqdm import tqdm


emotion_labels = ["joy", "sadness", "anger", "fear", "surprise", "disgust", "neutral"]
isbn = []
emotion_scores = {label : [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])


100%|██████████| 5197/5197 [04:41<00:00, 18.46it/s]


In [28]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [29]:
emotions_df.head()

,joy,sadness,anger,fear,surprise,disgust,neutral,isbn13
0,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612618,0.348285,0.942528,0.704422,0.887940,0.111690,0.252544,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150723,0.360707,0.251881,0.732686,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765,9780006280934


In [30]:
books = pd.merge(books, emotions_df, on="isbn13")
books.head(10)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,joy,sadness,anger,fear,surprise,disgust,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704422,0.887940,0.111690,0.252544
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150723,0.360707,0.251881,0.732686,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765
5,9780006380832,0006380832,Empires of the Monsoon,Richard Hall,"Africa, East",http://books.google.com/books/content?id=MuPEQ...,Until Vasco da Gama discovered the sea-route t...,1998.0,4.41,608.0,...,Empires of the Monsoon: A History of the India...,9780006380832 Until Vasco da Gama discovered t...,Nonfiction,0.232225,0.727174,0.051363,0.043376,0.621393,0.111690,0.271903
6,9780006470229,000647022X,The Gap Into Madness,Stephen R. Donaldson,"Hyland, Morn (Fictitious character)",http://books.google.com/books/content?id=4oXav...,A new-cover reissue of the fourth book in the ...,1994.0,4.15,743.0,...,The Gap Into Madness: Chaos and Order,9780006470229 A new-cover reissue of the fourt...,Fiction,0.538184,0.155855,0.747428,0.872565,0.712195,0.407999,0.078765
7,9780006472612,0006472613,Master of the Game,Sidney Sheldon,Adventure stories,http://books.google.com/books/content?id=TkTYp...,Kate Blackwell is an enigma and one of the mos...,1982.0,4.11,489.0,...,Master of the Game,9780006472612 Kate Blackwell is an enigma and ...,Nonfiction,0.064134,0.104007,0.404496,0.040564,0.549477,0.820282,0.234488
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,...,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction,0.300671,0.279481,0.915524,0.040564,0.840290,0.354459,0.135615
9,9780006483014,0006483011,The Once and Future King,Terence Hanbury White,Arthurian romances,http://books.google.com/books/content?id=Jx6Bv...,An omnibus volume of the author's complete sto...,1996.0,4.04,823.0,...,The Once and Future King,9780006483014 An omnibus volume of the author'...,Fiction,0.064134,0.177926,0.051363,0.040564,0.860372,0.111690,0.078765


In [32]:
books.to_csv('books_with_emotions.csv', index=False)